In [ ]:
from dataset import *

In [ ]:
storona = 30
dims = (storona,storona,storona)
class ConvLayer(nn.Module):
    def __init__(self,channels=[1,256],kernel_size=[9],stride=[1],print_=False):
        super(ConvLayer,self).__init__()
        self.layers=len(channels)-1
        self.print_=print_
        self.conv = nn.ModuleList([torch.nn.Conv3d(channels[l],channels[l+1],kernel_size[l],stride[l])
                                   for l in range(self.layers)])
#         self.conv = torch.nn.Conv3d(channels[0],channels[-1],kernel_size[0],stride[0])
#         for l in range(layers):
#             self.conv[l] = torch.nn.Conv3d(in_ch,out_ch[l],kernel_size[l],stride[l])
    def setPrint(print_):
        self.print_=print_
    def squash(self,x):
        if self.print_:
            print("shape before squash ",x.shape)
        norms = torch.norm(x,dim=1,keepdim=True)
#         print(norms.shape)
        x = ((norms/(1+norms**2))*x)
        if self.print_:
            print("norms.shape", norms.shape)
            print("shape after squash ",x.shape)

        return x
    def print_size(self,f):
        self.print_=f
    def forward(self,x):
        x = F.adaptive_avg_pool3d(x,dims)
#         x = F.avg_pool3d(x,3)
#         x = F.max_pool3d(x,3)
        for l in range(self.layers):
            print('Layer',l)
            if self.print_:
                print(f"shape before conv[{l}] ",x.shape)
            x = self.conv[l](x)
            if self.print_:
                print(f"shape after conv[{l}]",x.shape)
            x = self.squash(x)
#             x = F.leaky_relu(x)
        return x

In [ ]:
model = ConvLayer(channels=[1,256,128],kernel_size=[9,9],stride=[1,1],print_=True)


In [ ]:
for i in train_loader:
    print(i[0].shape)
    output = model(i[0])
    print("final output.shape",output.shape)
    break

In [ ]:


class PrimaryCapsules(nn.Module):
    def __init__(self,in_ch=256,out_caps_grids=32,out_ch_of_each_caps=8,kernel_size=9,stride=2):
        super(PrimaryCapsules,self).__init__()
        self.out_caps_grids=out_caps_grids
        self.out_ch_of_each_caps=out_ch_of_each_caps
        self.prim_caps = torch.nn.Conv3d(in_ch,out_caps_grids*out_ch_of_each_caps,kernel_size,stride)
    def reshape(self,x):
#         print('reshape prim')
        b,ch1,d1,d2,d3=x.shape
#         print(x.shape)
        x=x.permute(0,2,3,4,1)
#         print(x.shape)
        x=x.reshape(b,d1,d2,d3,self.out_caps_grids*self.out_ch_of_each_caps)
#         print(x.shape)
        return x
    def squash(self,x):
#         print('squash prim')
#         print(x.shape)
        norms = torch.norm(x,dim=-1,keepdim=True)
#         print(norms.shape)
        x = ((norms/(1+norms**2))*x)
#         print(x.shape)
        return x
    def forward(self,x):
        return self.squash(self.reshape(self.prim_caps(x)))
    
class DigitCaps(nn.Module):
    def __init__(self,input_grid=(6, 6, 6, 32),in_cap_dim=8,out_cap_dim=16,out_caps=3):
        super(DigitCaps,self).__init__()
        self.in_cap_dim=in_cap_dim
        self.num_caps = 1
        
        for i in input_grid:
            self.num_caps*=i
        self.W = torch.nn.Parameter(torch.randn(1, out_caps, self.num_caps, out_cap_dim, in_cap_dim))
        self.b = torch.nn.Parameter(torch.zeros(1, out_caps, self.num_caps,1,1),requires_grad=False)
    def reshape(self,x):
#         print('reshape digit')
#         print(x.shape)
        batch_size=x.shape[0]
        x=x.reshape(batch_size,1,-1,self.in_cap_dim,1)
        return x
    def squash(self,x):
        norms = torch.norm(x,dim=-1,keepdim=True)
        x = ((norms/(1+norms**2))*x)
        return x
    def forward(self,x):
#         print('forward Digit')
        batch_size=x.shape[0]
#         print('x',x.shape)
        u = self.reshape(x)
#         print('w',self.W.shape)
#         print('u',u.shape)
        
        uhat = torch.matmul(self.W,u)
#         print('uhat',uhat.shape)
        b = self.b

        for i in range(3):
#             print('iter ',i)
#             print('b',self.b.shape)
            c = torch.softmax(b,dim=2)
#             print('c',c.shape)
#             print(c.sum())
#             try:
            ahat= (uhat*c)
#             except:
#                 print('b',self.b.shape)

#                 print('uhat',uhat.shape)
#                 print('c',c.shape)

#             print('ahat',ahat.shape)
            a=ahat.sum(dim=2,keepdim=True).permute(0,1,2,4,3)
#             print('a',a.shape)
#             print('uhat',uhat.shape)
            adotuhat=torch.matmul(a,uhat).mean(dim=0,keepdim=True)
#             print('adotuhat.shape',adotuhat.shape)
#             print("adotuhat.mean(dim=0,keepdim=True).shape",adotuhat.mean(dim=0,keepdim=True).shape)
#             print("b.shape",b.shape)
            b = b+adotuhat
            if self.training:
                self.b=torch.nn.Parameter(b,requires_grad=False)
        x = self.squash(a).squeeze().squeeze()
        return x

In [ ]:
for batch_id, (data, target) in enumerate(train_loader,1):
    data = data.cuda()
    print(data.shape)
    input_shape=data.shape

#!!!!!!!!!!!!!!!!!!!!
    out_ch_CONV=[1,128]
    kernel_size_CONV=[15]
    stride_CONV=[1]

    
    model1 = ConvLayer(
        channels=out_ch_CONV,
        kernel_size=kernel_size_CONV,
        stride=stride_CONV,
        print_=True
    ).cuda()
    data = model1(data)
#     print(data.shape)
    model1.print_size(False)

#!!!!!!!!!!!!!!!!!!!!
    in_ch_PRIM=out_ch_CONV[-1]
    out_caps_grids_PRIM=100
    out_ch_of_each_caps_PRIM=5
    kernel_size_PRIM=7
    stride_PRIM=3
    model2 = PrimaryCapsules(
        in_ch=in_ch_PRIM,
        out_caps_grids=out_caps_grids_PRIM,
        out_ch_of_each_caps=out_ch_of_each_caps_PRIM,
        kernel_size=kernel_size_PRIM,
        stride=stride_PRIM
    ).cuda()
    
    data = model2(data)
    print(data.shape)
    
#!!!!!!!!!!!!!!!!!!!!
    input_grid_DIGIT=data.shape[1:-1]
    in_cap_dim_DIGIT=data.shape[-1]
    out_cap_dim_DIGIT=1
    out_caps_DIGIT=len(set(diagnos.values()))
    model3 = DigitCaps(
        input_grid=input_grid_DIGIT,
        in_cap_dim=in_cap_dim_DIGIT,
        out_cap_dim=out_cap_dim_DIGIT,
        out_caps=out_caps_DIGIT
    ).cuda()
    
    data = model3(data)
    print(data.shape)
#     print(torch.norm(data,dim=-1,keepdim=True).squeeze().squeeze().squeeze().shape)
    break

In [ ]:
models = [
    model1,
    model2,
    model3
]

In [ ]:
total=0
for i,model in enumerate(models):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    print('Number of parameters in model ',i,params)
    total+=params
print('Number of total parameters model',total) 

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder,self).__init__()
#         self.l1=ConvLayer()
#         self.l2=PrimaryCapsules()
#         self.l3=DigitCaps()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
#         self.model4 = model4
    def forward(self,data):
        data = data.cuda()
        data = self.model1(data)
#         print(data.shape)
        data = self.model2(data)
#         print(data.shape)
        data = self.model3(data)
#         print(data.shape)
#         data = self.model4(data)
#         print(data.shape)
#         output = torch.norm(data,dim=-1,keepdim=True).squeeze().squeeze()
#         print('encoder output.shape',output.shape)
        return data
# class Decoder(nn.Module):
#     def __init__(self,l0=16,l1=512,l2=4024,l3=902629):
#         super(Decoder,self).__init__()
#         self.fc1=nn.Linear(l0,l1)
#         self.fc2=nn.Linear(l1,l2)
#         self.fc3=nn.Linear(l2,l3)
# #         self.fc4=nn.Linear(l3,l4)

#     def forward(self,x):
# #         print(x.shape)
#         x = self.fc1(x)
# #         print(x.shape)
#         x=f.relu(x)
#         x = self.fc2(x)
#         x=f.relu(x)
# #         print(x.shape)
#         x=self.fc3(x)
# #         x=f.relu(x)
# #         x=self.fc4(x)
#         x=torch.sigmoid(x)
# #         print(x.shape)
#         return x

class CapsNet3D(nn.Module):
    def __init__(self):
        super(CapsNet3D,self).__init__()
        self.enc = Encoder()
#         self.dec = Decoder()
        self.encloss=nn.BCELoss()
#         self.encloss=nn.CrossEntropyLoss()
    def margin_loss(self,pred_prob,target):

#         print('pred_prob.shape',pred_prob.shape)
#         print('pred_prob[:5]',pred_prob[:5])
#         print('target.shape',target.shape)
#         print('target[:5]',target[:5])
        
        _, max_length_indices = pred_prob.max(dim=1)
#         print('_.shape',_.shape)
#         print('_[:5]',_[:5])
#         print('max_length_indices.shape',max_length_indices.shape)
#         print('max_length_indices[:5]',max_length_indices[:5])
        masked= Variable(torch.sparse.torch.eye(n_classes_by_val),requires_grad=False)
#         masked = torch.eye(n_classes)
        if USE_CUDA:
            masked=masked.cuda()
        
#         print('masked.shape',masked.shape)
#         print('masked',masked)
#         masked = masked.index_select(dim=0, index=max_length_indices.data)
        masked = masked.index_select(dim=0, index=target)

        Tk = masked
#         print("Tk.shape",Tk.shape)
#         print("Tk[0:5]",Tk[0:5])
        left = Tk*F.relu(0.9-pred_prob)
#         print('left.shape',left.shape)
        right = 0.5*(1-Tk)*F.relu(pred_prob-0.1)
#         print('right.shape',right.shape)
        total =(left+right).mean()
#         print('total.shape',total.shape)
        return total
    def loss(self,pred_prob,target_class,target_hot, target_recon=None,decout=None):
#         pred_prob =torch.norm(encout,dim=-1,keepdim=False)
        encoder_loss = self.encloss(pred_prob,target_hot)
#         encoder_loss = self.encloss(pred_prob,target_class)
#         print(target_class[0])
#         print(pred_prob[0])
        margin = self.margin_loss(pred_prob,target_class)
#         decoder_loss = ((target_recon-decout)**2).sum()
#         total = margin
#         total = encoder_loss,margin
#         print(total, typeof)
#         return encoder_loss,decoder_loss
#         return margin,decoder_loss
#         return encoder_loss
#         return margin
        return margin+encoder_loss
    
    def fetch(self,encout):
#         print('fetching')
#         print('fetch encout.shape',encout.shape)
        if len(encout.shape)==2:
            encout=encout.unsqueeze(0)
        pred_prob =torch.norm(encout,dim=-1,keepdim=False)#.squeeze()
#         print(' fetch prob',pred_prob.shape)
        if len(pred_prob.shape)==1:
            pred_prob=pred_prob.reshape(1,-1)
        _, max_length_indices = pred_prob.max(dim=1)
#         print('_.shape',_.shape)
#         print('_[:10]',_[:10])
#         print('max_length_indices.shape',max_length_indices.shape)
#         print('max_length_indices[:10]',max_length_indices[:10])
        masked= Variable(torch.sparse.torch.eye(n_classes_by_val),requires_grad=False)

#         masked = torch.sparse.torch.eye(n_classes)
        if USE_CUDA:
            masked=masked.cuda()
        
#         print('masked.shape',masked.shape)
#         print('masked',masked)
        masked = masked.index_select(dim=0, index=max_length_indices.data)
#         print('masked.shape',masked.shape)
#         print('masked',masked)
#         print('encout.shape',encout.shape)

        decin =torch.matmul(masked[:, None,:],encout)
#         print('decin.shape',decin.shape)
        decin = decin.view(encout.size(0), -1)
#         print('decin',decin)
#         print(decin[0,:])
#         print(encout[0,max_length_indices[0],:])
        return pred_prob,decin
        
    def forward(self,x):
        encout=self.enc(x)
        pred_prob,decin=self.fetch(encout)
#         decout=self.dec(decin).reshape(-1,1,28,28)
#         print(decout.shape)
#         return pred_prob,decout
        return pred_prob


In [ ]:

model = CapsNet3D().cuda()
loss_fn=model.loss
# loss_fn = nn.SoftMarginLoss()
# loss_fn = nn.BCELoss()
# loss_fn = nn.BCEWithLogitsLoss()
# subjZ=[0]*n_classes_by_val

# for i,v in enumerate(diagnos.values()):
#     subjZ[v]+=n_subj_train[i]
# weights = [1/i for i in subjZ]
# norm = 0
# for i in weights:
#     norm+=i**2
# norm = norm**0.5
# weights=[i/(norm) for i in weights]

# loss_fn=nn.CrossEntropyLoss(weight=torch.tensor(weights).cuda())

